<a href="https://colab.research.google.com/github/chekfung/cross_layer_final_project/blob/main/training_quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Source Code

## Imports Needed Throughout the Project

In [1]:
# All Imports 
import sys
import numpy as np
import copy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from google.colab import files

# argument parser
import easydict

## Get and Download Datasets

In [2]:
# MNIST Dataset (Images and Labels)
train_set = dsets.FashionMNIST(
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

test_set = dsets.FashionMNIST(
    root = './data/FashionMNIST',
    train = False,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()                                 
    ])
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/FashionMNIST/raw



## More Helper Code for Training and Testing Accuracy

In [3]:
def train_model(model, criterion, optimizer, train_loader):
    print("---Training started")
    # Training the Model
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            # Load Images into GPU
            images = images.cuda()
            labels = Variable(labels).cuda()

            # Forward + Backward + Optimize
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            L1norm = model.parameters()
            arr = []

            # Calculate L1 Norm (if included in hyperparameters)
            if args.L1norm == True:
                for name,param in model.named_parameters():
                    if 'weight' in name.split('.'):
                        arr.append(param)

                L1loss = 0
                for Losstmp in arr:
                    L1loss = L1loss+Losstmp.abs().mean()

                if len(arr) > 0:
                    loss = loss+L1loss/len(arr)

            # Optimizer Step, Propagate Loss backwards
            loss.backward()
            optimizer.step()

            if (i + 1) % 600 == 0:
                print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                        % (epoch + 1, num_epochs, i + 1,
                        len(train_set) // batch_size, loss.data.item()))


# Gets accuracy given dataset as well as total test loss
def get_acc(model, criterion, test_loader):
    correct = 0
    total = 0

    for images, labels in test_loader:
        images = images.cuda()
        labels = labels.cuda()
        outputs = model(images)
        testloss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        break

    return ((100 * correct / total), testloss)

## FP32 Model 

In [4]:
class MyConvNet_FP32(nn.Module):
    def __init__(self, args):
        super(MyConvNet_FP32, self).__init__()
        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn1   = nn.BatchNorm2d(16)
        self.act1  = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        # Layer 2
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2   = nn.BatchNorm2d(32)
        self.act2  = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        # Layer 3
        self.lin2  = nn.Linear(7*7*32, 10)

    def forward(self, x):
        # Layer 1
        c1 = self.conv1(x)
        b1  = self.bn1(c1)
        a1  = self.act1(b1)
        p1  = self.pool1(a1)

        # Layer 2
        c2  = self.conv2(p1)
        b2  = self.bn2(c2)
        a2  = self.act2(b2)
        p2  = self.pool2(a2)

        # Flatten and Layer 3
        flt = p2.view(p2.size(0), -1)
        out = self.lin2(flt)
        return out
  
# model = MyConvNet(args)
# model = model.cuda()

## Quantization Helper Code

In [31]:
def simple_quantize_val(val, scale_factor, min_val, max_val):
  value = torch.round(val / scale_factor)

  if (value < min_val):
    value = min_val

  if (value > max_val):
    value = max_val

  return (value * scale_factor)

def fixed_point_quantize_val(val, num_bits, fractional_bits):
  integer_bits = num_bits - fractional_bits - 1 # Subtract one for sign bit
  smallest_step_size = 1 / np.power(2, fractional_bits)
  largest_number = (np.power(2, integer_bits) - 1) + ((np.power(2, fractional_bits)-1) * smallest_step_size)
  smallest_number = -1 * np.power(2, integer_bits)

  value = torch.round(val / smallest_step_size) * smallest_step_size

  if (value < smallest_number):
    value = smallest_number

  if (value > largest_number):
    value = largest_number

  return value

# Perhaps slightly optimized version?
def fixed_point_quantize_faster(val, smallest_step_size, largest_number, smallest_number):

  # Perform Pseudo Quantization
  value = torch.round(val / smallest_step_size) * smallest_step_size

  # Clamp Values
  if (value < smallest_number):
    value = smallest_number

  if (value > largest_number):
    value = largest_number

  return value

# For flattened and output tensors
def quantize_tensor_2d(tens, step_size, largest_num, smallest_num):
  shape = tens.shape

  # Go through entire tensor and quantize in place
  for i in range(shape[0]):
    for j in range(shape[1]):
      tens[i][j] = fixed_point_quantize_faster(tens[i][j], step_size, largest_num, smallest_num)

# Literally for everything else
def quantize_tensor_4d(tens, step_size, largest_num, smallest_num):
  shape = tens.shape

  for i in range(shape[0]):
    for j in range(shape[1]):
      for k in range(shape[2]):
        for l in range(shape[3]):
          tens[i][j][k][l] = fixed_point_quantize_faster(tens[i][j][k][l], step_size, largest_num, smallest_num)

## Quantization Model

In [34]:
# Assume that we always 
class MyConvNet_FIXED_POINT(nn.Module):
    def __init__(self, args, num_bits, num_fractional_bits):
        super(MyConvNet_FIXED_POINT, self).__init__()

        # Fixed Point Parameters
        self.fp_bits = num_bits
        self.sign_bit = 1
        self.integer_bits = (num_bits - 1 - num_fractional_bits)
        self.fractional_bits = num_fractional_bits

        # Fixed Point Computed Values for Quantization
        self.smallest_step_size = 1 / np.power(2, num_fractional_bits)
        self.largest_num_representable = (np.power(2, self.integer_bits) - 1) + ((np.power(2, self.fractional_bits)-1) * self.smallest_step_size)
        self.smallest_num_representable = -1 * np.power(2, self.integer_bits)

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn1   = nn.BatchNorm2d(16)
        self.act1  = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        # Layer 2
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.bn2   = nn.BatchNorm2d(32)
        self.act2  = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        # Layer 3
        self.lin2  = nn.Linear(7*7*32, 10)


    # AUGMENT FORWARD PASS. forward pass not quantizes every single time we go through
    def forward(self, x):
        # Layer 1
        c1 = self.conv1(x)
        quantize_tensor_4d(c1, self.smallest_step_size, self.largest_num_representable, self.smallest_num_representable)
        b1  = self.bn1(c1)
        quantize_tensor_4d(b1, self.smallest_step_size, self.largest_num_representable, self.smallest_num_representable)
        a1  = self.act1(b1)
        quantize_tensor_4d(a1, self.smallest_step_size, self.largest_num_representable, self.smallest_num_representable)
        p1  = self.pool1(a1)
        quantize_tensor_4d(p1, self.smallest_step_size, self.largest_num_representable, self.smallest_num_representable)

        # Layer 2
        c2  = self.conv2(p1)
        quantize_tensor_4d(c2, self.smallest_step_size, self.largest_num_representable, self.smallest_num_representable)
        b2  = self.bn2(c2)
        quantize_tensor_4d(b2, self.smallest_step_size, self.largest_num_representable, self.smallest_num_representable)
        a2  = self.act2(b2)
        quantize_tensor_4d(a2, self.smallest_step_size, self.largest_num_representable, self.smallest_num_representable)
        p2  = self.pool2(a2)
        quantize_tensor_4d(p2, self.smallest_step_size, self.largest_num_representable, self.smallest_num_representable)

        # Flatten and Layer 3
        flt = p2.view(p2.size(0), -1)
        out = self.lin2(flt)
        quantize_tensor_2d(out, self.smallest_step_size, self.largest_num_representable, self.smallest_num_representable)
        return out

# Test Code Here


In [39]:
args = easydict.EasyDict({
        "batch_size": 1,
        "epochs": 3,
        "lr": 0.001,
        "enable_cuda" : True,
        "L1norm" : False,
        "simpleNet" : True,
        "activation" : "relu", #relu, tanh, sigmoid
        "train_curve" : True, 
        "optimization" :"SGD"
})

# Hyper Parameter for FashionMNIST
input_size = 784
num_classes = 10
num_epochs = args.epochs
batch_size = args.batch_size
learning_rate = args.lr

# Dataset Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset = train_set, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle = False)

In [38]:
# Declare Model
model = MyConvNet_FIXED_POINT(args, 8, 5).cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) 

# Training? Commented out for now so that I can just make sure that the forward quantization works.
# train_model(model, criterion, optimzer, train_loader)

# Testing ARC
test_acc, test_loss = get_acc(model, criterion, test_loader)
print("Test Accuracy: {}".format(test_acc))
print("Test Loss: {}".format(test_loss))

Test Accuracy: 0.0
Test Loss: 2.570774555206299
